# Test plane-based registration

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import os

import planeslam.io as io
from planeslam.general import plot_3D_setup, color_legend
from planeslam.scan import pc_to_scan
from planeslam.registration import get_correspondences

Read in airsim LiDAR and pose data

In [2]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(),'..', '..', 'data', 'airsim', 'blocks_20_samples_1', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

In [3]:
# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(),'..', '..', 'data', 'airsim', 'blocks_20_samples_1', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

Extract planes 

In [4]:
num_scans = len(PC_data)
scans = num_scans * [None]
scans_transformed = num_scans * [None]
for i in range(num_scans):
    scans[i] = pc_to_scan(PC_data[i])

In [5]:
# ax = plot_3D_setup(PC_data[0])
# cmap = cm.get_cmap('viridis')
# cmap_idxs = int(len(cmap.colors) / num_scans) * np.arange(num_scans)
# colors = np.asarray(cmap.colors)[cmap_idxs,:]

# for i, s in enumerate(scans):
#     s.plot(ax, color=colors[i])

Get correspondences

In [5]:
# # Plot first 2 scans
# ax = plot_3D_setup(PC_data[0])
# for i, scan in enumerate(scans[0:2]):
#     scan.plot(ax, show_normals=True)

# # Color legend
# num_colors = max([len(s.planes) for s in scans[0:2]])
# color_legend(ax, num_colors)

In [6]:
P = scans[0]
Q = scans[1]
correspondences = get_correspondences(P, Q)

1 1
3 3
5 4


In [ ]:
def extract_corresponding_features(P, Q):
    """Extract corresponding normals and distances for source and target scans

    Parameters
    ----------
    P : Scan
        Source scan
    Q : Scan
        Target scan
    
    """
    correspondences = get_correspondences(P, Q)

    for c in correspondences:

    N = len(scan.planes)
    n = np.empty((3,N))
    rho = np.empty((N,1))

    for i in range(N):
        normal = scan.planes[i].normal
        center = scan.planes[i].center
        n[:,i][:,None] = normal
        rho[i] = np.dot(normal.flatten(), center)

    n = n.reshape((3*N,1))

    return n, rho

In [ ]:
# Extract normals and distances


Test Gauss-Newton registration

In [24]:
from planeslam.geometry.util import axis_angle_to_rot_mat


def R_from_q(q):
    """Extract rotation matrix from parameterized transformation q

    Parameters
    ----------
    q : np.array (6 x 1)
        Parameterized transformation

    Returns
    -------
    np.array (3 x 3)
        Rotation matrix
    
    """
    theta = np.linalg.norm(q[3:])  # NOTE: need to wrap?
    if theta == 0:
        return np.eye(3)
    else:
        u = q[3:] / theta 
        return axis_angle_to_rot_mat(u, theta)


def transform_normals(n, q):
    """Transform normals

    n(q) = [...,Rn_i,...]

    Parameters
    ----------
    n : np.array (3N x 1)
        Stacked vector of normals
    q : np.array (6 x 1)
        Parameterized transformation

    Returns
    -------
    np.array (3N x 1)
        Transformed normals

    """
    assert len(n) % 3 == 0, "Invalid normals vector, length should be multiple of 3"
    N = int(len(n) / 3)

    # Extract rotation matrix R from q 
    R = R_from_q(q)

    # Apply R to n
    n = n.reshape((3, N), order='F')
    n = R @ n
    n = n.reshape((3*N, 1), order='F')
    return n


def transform_dists(rho, n, q):
    """Transform distances

    rho(q) = [...,(R n_i)^T t + rho_i,...]

    Parameters
    ----------
    rho : np.array (N x 1)
        Stacked vector of distances
    n : np.array (3N x 1)
        Stacked vector of normals
    q : np.array (6 x 1)
        Parameterized transformation

    Returns
    -------
    np.array (N x 1)
        Transformed distances

    """
    t = q[:3]
    Rn = transform_normals(n, q)
    Rn = Rn.reshape((-1,3))
    return rho + Rn @ t


def residual(n_s, rho_s, n_t, rho_t, q):
    """Residual for Gauss-Newton

    Parameters
    ----------
    n_s : np.array (3N x 1)
        Stacked vector of source normals
    rho_s : np.array (N x 1)
        Stacked vector of source distances
    n_t : np.array (3N x 1)
        Stacked vector of target normals
    rho_t : np.array (N x 1)
        Stacked vector of target distances
    q : np.array (6 x 1)
        Parameterized transformation

    Returns
    -------
    np.array (4N x 1)
        Stacked vector of plane-to-plane error residuals
    
    """
    n_q = transform_normals(n_s, q)

    # Transform distances
    t = q[:3]
    rho_q = rho_s + n_q.reshape((-1,3)) @ t 

    return np.vstack((n_q - n_t, rho_q - rho_t))
    


def jacobian(n_s, q):
    """Jacobian for Gauss-Newton
    
    """

In [28]:
# Get n's and rho's
n_s, rho_s = n_rho_from_scan(scans[0])
n_t, rho_t = n_rho_from_scan(scans[1])

In [19]:
t = np.array([10, 0, 0])[:,None]
u = np.array([1, 0, 0])[:,None]
theta = np.pi/2
q = np.vstack((t, theta*u))

In [29]:
residual(n_s, rho_s, n_t, rho_t, q)

ValueError: operands could not be broadcast together with shapes (18,1) (21,1) 

In [20]:
transform_normals(n_s, q)

array([[-4.55292970e-04],
       [ 9.99999818e-01],
       [-1.31836837e-04],
       [-6.59782071e-03],
       [-2.25731767e-02],
       [ 9.79675056e-03],
       [ 2.19370533e-04],
       [-5.81487471e-04],
       [-9.99999965e-01],
       [ 9.99978234e-01],
       [ 9.99687449e-01],
       [-9.25331724e-01],
       [-9.99999872e-01],
       [-1.57771361e-04],
       [-2.29716127e-04],
       [ 1.13268707e-06],
       [ 1.07423204e-02],
       [-3.79010715e-01]])

In [ ]:
# Initial transformation
t = np.zeros((3,1))
u = np.array([1, 0, 0])[:,None]
theta = 0
q = np.vstack((t, theta*u))


# Gauss-Newton
n_iters = 5

for i in range(n_iters):
    J = jacobian(n_s, q)
    q = q - np.linalg.inv(J.T @ J) @ J.T @ residual(n_s, rho_s, n_t, rho_t, q)